# **Data Collection and Preparation**

## Objectives

* Fetch data from Kaggle and prepare for further processing

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Generate Dataset: datasets/codeinstitute/cherry-leaves

## Additional Comments

* No additional comments

---

# Import packages

In [9]:
%pip install -r /workspace/PP5-mildew-detection-in-cherry-leaves/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy
import os

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()


In [5]:
import os
current_dir = os.getcwd()
current_dir


'/workspace/PP5-mildew-detection-in-cherry-leaves/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [6]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")


You set a new current directory


Confirm the new current directory

In [7]:
current_dir = os.getcwd()
current_dir

'/workspace/PP5-mildew-detection-in-cherry-leaves'

## Install Kaggle

In [8]:
# install kaggle package
%pip install kaggle==1.5.12

Note: you may need to restart the kernel to use updated packages.


---

Run the cell below to change the kaggle configuration directory to the current working directory and set permissions for the kaggle authentication JSON

In [9]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set Kaggle Dataset and Download it

In [12]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 98%|█████████████████████████████████████▎| 54.0M/55.0M [00:02<00:00, 29.6MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 23.2MB/s]


Unzip the downloaded file, and delete the zip file

In [13]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

---

### Data Cleaning

#### Check and remove non-image files

In [4]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [6]:
remove_non_image_file(my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves')


Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


## Split train validation test set 

In [7]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

- The training set is divided into 0.70 ratio data
- The validation set is divided into a 0.10 ratio data
- The test set is divided into a 0.20 raitio data

In [8]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry-leaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )


---

# Push to Repo

In [8]:
import os
import shutil

# Step 1: Create the directory
try:
    os.makedirs(name='outputs/datasets/collection')  # Create outputs/datasets/collection folder
except Exception as e:
    print(e)

# Step 2: Copy files from the input directory to the output directory
def copy_files(src, dst):
    try:
        if os.path.exists(src):
            shutil.copytree(src, dst, dirs_exist_ok=True)
            print(f"Copied files from '{src}' to '{dst}'")
        else:
            print(f"Source directory '{src}' does not exist")
    except Exception as e:
        print(f"Error copying files: {e}")

if __name__ == "__main__":
    input_directory = 'inputs/cherry-leaves_dataset'  # Input directory
    output_directory = 'outputs/datasets/collection'  # Output directory

    # Copy files from input directory to output directory
    copy_files(input_directory, output_directory)



Copied files from 'inputs/cherry-leaves_dataset' to 'outputs/datasets/collection'
